In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.ensemble import GradientBoostingClassifier

## Импортируем csv

In [95]:
data = pd.read_csv('train_dataset_train.csv', sep=',')
# data=data.head(150000)
# features=data.drop(labels=["time_to_under","labels"])


## Предобработка
Удаляем бесполезные колонки и разбивка на тренировочную и тестовую части


In [96]:
data = data.drop(['id', 'ticket_id', 'entrance_id', 'station_id', 'line_id', 'entrance_nm'], axis=1)

In [97]:
data['pass_dttm'] = pd.to_datetime(data['pass_dttm'])

In [98]:
data.isnull().sum()

ticket_type_nm    0
station_nm        0
line_nm           0
pass_dttm         0
time_to_under     0
label             0
dtype: int64

In [99]:
data['hour'] = data['pass_dttm'].dt.hour
data['time_of_day'] = data['hour'].apply(
    lambda hour: 'day' if hour>=10 and hour<=16 else (
        'night' if hour>=0 and hour<=4 else (
            'morning' if hour>=4 and hour<=9 else 'evening')))

data['weekday'] = data['pass_dttm'].dt.weekday
data['is_weekend'] = data['weekday'].apply(
    lambda weekday: 1 if weekday>4 else 0)

data = data.drop(columns=['pass_dttm', 'hour', 'weekday'])


## Категориальные признаки (тип==обджект)

In [100]:
obj_feat = list(data.loc[:, data.dtypes=='object'].columns.values)
print(obj_feat)

['ticket_type_nm', 'station_nm', 'line_nm', 'time_of_day']


In [101]:
for feature in obj_feat:
    data[feature] = pd.Series(data[feature], dtype='category')

## Разбиваем на тренировочную и тестововую части

In [102]:
X = data.drop(columns=['time_to_under', 'label'])
y = data['time_to_under']

X_train_for_regression, X_test_for_regression, y_train_for_regression, y_test_for_regression = train_test_split(X, y, test_size=0.1)
X_train_for_regression, X_test_for_regression, y_train_for_regression, y_test_for_regression = train_test_split(X_test_for_regression, y_test_for_regression, test_size=0.3)

## Регрессор

In [103]:
gbm = lgb.LGBMRegressor(objective='regression')

params = {
    'num_leaves': range(1, 102, 10),
    'n_estimators': range(1, 102, 10)
}
grid = GridSearchCV(gbm, params, cv=5)
grid.fit(X_train_for_regression, y_train_for_regression)

gbm = grid.best_estimator_
print(grid.best_params_)

gbm.fit(X_train_for_regression, y_train_for_regression)

y_predict_for_regression = gbm.predict(X_test_for_regression, num_iteration=gbm.best_iteration_)

D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
55 fits failed out of a total of 605.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\lib\site-packages\lightgbm\sklearn.py", line 895, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score,
  File "D:\anaconda\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "D:\anaconda\lib\site-packages\lightgbm\engine.py", line 271, in train
  

{'n_estimators': 51, 'num_leaves': 11}


In [109]:
X = data.drop(columns=['time_to_under', 'label'])
y = data['label']

X_train_for_classifier, X_test_for_classifier, y_train_for_classifier, y_test_for_classifier = train_test_split(X, y, test_size=0.1)

X_train_for_classifier, X_test_for_classifier, y_train_for_classifier, y_test_for_classifier = train_test_split(X_test_for_classifier, y_test_for_classifier, test_size=0.3)

## Классифаер

In [113]:
gbm = lgb.LGBMClassifier(objective='multiclass', max_depth=20, n_estimators=10)
gbm.fit(X_train_for_classifier, y_train_for_classifier)
y_predict_for_classifier = gbm.predict(X_test_for_classifier)

#classifier=GradientBoostingClassifier(n_iter_no_change=10)
#classifier.fit(X_test_for_classifier,np.ravel(y_test_for_classifier))
#y_predict_for_classifier=classifier.predict(X_test_for_classifier)

In [114]:
result = 0.5*r2_score(y_test_for_regression, y_predict_for_regression) + 0.5*recall_score(y_test_for_classifier, y_predict_for_classifier, average='micro')
print(result)

0.2593380304514765
